In [ ]:
# 그래프, 수학 기능 추가
# Add graph and math features
import pylab as py
import numpy as np
import numpy.linalg as nl
# 기호 연산 기능 추가
# Add symbolic operation capability
import sympy as sy

In [ ]:
sy.init_printing()

# 전진 오일러법: 고차 미분 방정식<br>Forward Euler Method: Higher Order ODE

## 1계 미분방정식의 예<br>An example of a first order ODE

$$
\left\{
    \begin{align}
        2 \frac{d}{dt}x(t)+ x(t)&=0 \\
        x(0)&=x_0 \\
    \end{align}
\right.
$$

위 1계 미분방정식의 경우 다음과 같이 근사해를 구할 수 있었다.<br>We could find an approximate solution for a first order ordinary differential equation as follows.

In [ ]:
a_0, a_1 = 2.0, 1.0

def dx_dt(x, t):
    return - a_1 * x / a_0

In [ ]:
def forward_euler(f, t_array, x_0):
    time_list = [t_array[0]]
    result_list = [x_0]

    x_i = x_0

    for k, t_i in enumerate(t_array[:-1]):
        # time step
        delta_t = t_array[k+1] - t_array[k]

        # slope
        s_i = f(x_i, t_i)

        # x[i + 1]
        x_i_plus_1 = x_i + s_i * delta_t

        time_list.append(t_array[k+1])
        result_list.append(x_i_plus_1)
        
        x_i = x_i_plus_1

    return time_list, result_list



In [ ]:
# Time step interval
delta_t = 0.1

# Time array
t_sec_array = np.arange(0, 6 + delta_t*0.5, delta_t)

# Initial state
x_0 = 4.5

# *** ODE solver ***
t_01, x_01 = forward_euler(dx_dt, t_sec_array, x_0)


In [ ]:
def exact(t):
    return x_0 * py.exp((-a_1 / a_0) * t)


In [ ]:
py.plot(t_01, x_01, '.-', label='$\Delta t=0.1$')

# Indicate the exact solution
exact_x_array = exact(t_sec_array)
py.plot(t_sec_array, exact_x_array, label='exact')

py.axis('equal')

py.xlabel('t(sec)')
py.ylabel('x(m)')

py.legend(loc=0)
py.grid(True)


그러나 앞으로 만나게 될 미분 방정식은 더 높은 차수의 미분을 포함할 수 있다.<br>However, in the future, we may meet differential equations including higher order differentiation.

## 2계 비선형 상미분 방정식<br>Second order Nonlinear Ordinary Differential Equation

예를 들어 단진자가 어떻게 움직이는지 다음과 같이 묘사할 수 있다.<br>
For example, we can describe how a simple pendulum moves as follows.<br>
Ref : Wikipedia contributors, 'Pendulum (mathematics)', Wikipedia, The Free Encyclopedia, 2 June 2018, 13:28 UTC, <https://en.wikipedia.org/w/index.php?title=Pendulum_(mathematics)&oldid=844080803> [accessed 5 August 2018]

$$
\frac{d^2\theta}{dt^2} + \frac{g}{l}sin\theta = 0
$$

위 상미분 방정식은 $\theta$의 2계 미분과 $sin\theta$ 를 포함하고 있다.<br>The ordinary differential equation above includes a second order derivative of $\theta$ and $sin\theta$.

여기서 $sin\theta$ 는 비선형으로, 위 해당 식은 2계 비선형 상미분 방정식이다.<br>Here $sin\theta$ is nonlinear; thus the equation above is a second order nonlinear ordinary differential equation.

해당 방정식을 풀기 위해 다음과 같이 $x_0$, $x_1$ 와 같은 상태변수를 도입하자.<br>To solve the equation, let's introduce state variables $x_0$ and $x_1$ as follows.

$$
\begin{cases}
    x_0 = \theta\\
    x_1 = \frac{d\theta}{dt}  = \frac{d}{dt}x_0\\
\end{cases}
$$

방정식의 각 항을 상태변수로 다시 써 보자.<br>Let's rewrite each term of the equation using the state variables.

$$
\begin{cases}
    sin \theta = sin x_0\\
    \frac{d^2\theta}{dt^2} = \frac{d}{dt} \frac{d\theta}{dt}= \frac{d}{dt} x_1\\
\end{cases}
$$

다시 방정식에 대입해 보자.<br>Let's substitute back to the equation.

$$
\frac{dx_1}{dt} + \frac{g}{l}sinx_0 = 0 \\
\frac{dx_1}{dt} =- \frac{g}{l}sinx_0
$$

$x_0$와 $x_1$의 미분을 살펴 보자.<br>Let's take a look at the derivatives of $x_0$ and $x_1$.

$$
\begin{pmatrix}
    \frac{dx_0}{dt}\\
    \frac{dx_1}{dt}
\end{pmatrix} 
=
\begin{pmatrix}
    x_1\\
    -\frac{g}{l}sinx_0
\end{pmatrix} 
$$

이를 python 함수로 구현해 보자.<br>Let's implement this in a python function.

In [ ]:
g_mpsps = 9.8
l_m = 1

def pendulum_NL(x, t):
    """
    x: column vector of theta and d(theta)/dt
    t: time value
    """
    
    return np.array([x[1], (-g_mpsps/l_m)*np.sin(x[0])])


전진 오일러법을 적용해 보자.<br>Let's apply the Forward Euler Method.

In [ ]:
# Time step interval
delta_t = 0.1

# Time array
t_sec_array = np.arange(0, 6 + delta_t*0.5, delta_t)

# Initial state
x_0 = np.array([np.deg2rad(10), 0])

# *** ODE solver ***
t_01, x_01 = forward_euler(pendulum_NL, t_sec_array, x_0)


In [ ]:
py.plot(t_01, x_01, '.-', label='$\Delta t=0.1$')

py.axis('equal')

py.xlabel('t(sec)')
py.ylabel('x(m)')

py.legend(loc=0)
py.grid(True)


위 근사해는 불안정해 보인다. $\Delta t$를 줄여 보자.<br>The approximate solution looks unstable. Let's make $\Delta t$ smaller.

In [ ]:
# Time step interval
delta_t = 0.001

# Time array
t_sec_array = np.arange(0, 6 + delta_t*0.5, delta_t)

# Initial state
x_0 = np.array([np.deg2rad(10), 0])

# *** ODE solver ***
t_01, x_01 = forward_euler(pendulum_NL, t_sec_array, x_0)


In [ ]:
py.plot(t_01, x_01, label=f'$\Delta t={delta_t}$')

py.axis('equal')

py.xlabel('t(sec)')
py.ylabel('x(m)')

py.legend(loc=0)
py.grid(True)
